In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import os

In [2]:
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224, 224), transforms.InterpolationMode.BILINEAR),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((224,224), transforms.InterpolationMode.BILINEAR),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [4]:
data_dir = 'train_val_plots'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'val']}
dataloaders = {x: DataLoader(image_datasets[x], batch_size=32, shuffle=True, num_workers=4) for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

In [5]:
model = models.alexnet(pretrained=True)
num_ftrs = model.classifier[6].in_features
model.classifier[6] = nn.Linear(num_ftrs, len(class_names))
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

/home/cybercricetus/anaconda3/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/cybercricetus/anaconda3/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [6]:
num_epochs = 25
best_model_wts = model.state_dict()
best_acc = 0.0

for epoch in range(num_epochs):
    print(f'Epoch {epoch}/{num_epochs - 1}')
    print('-' * 10)
    
    for phase in ['train', 'val']:
        if phase == 'train':
            model.train()
        else:
            model.eval()
        
        running_loss = 0.0
        running_corrects = 0
        
        for inputs, labels in dataloaders[phase]:
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            optimizer.zero_grad()
            
            with torch.set_grad_enabled(phase == 'train'):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)
                
                if phase == 'train':
                    loss.backward()
                    optimizer.step()
            
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
        
        epoch_loss = running_loss / dataset_sizes[phase]
        epoch_acc = running_corrects.double() / dataset_sizes[phase]
        
        print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')
        
        if phase == 'val' and epoch_acc > best_acc:
            best_acc = epoch_acc
            best_model_wts = model.state_dict()
    
    print()

print('Training complete')
print(f'Best val Acc: {best_acc:.4f}')

Epoch 0/24
----------
train Loss: 0.6408 Acc: 0.6154
val Loss: 0.4635 Acc: 1.0000

Epoch 1/24
----------
train Loss: 0.5441 Acc: 0.8462
val Loss: 0.3305 Acc: 1.0000

Epoch 2/24
----------
train Loss: 0.4796 Acc: 0.7692
val Loss: 0.1712 Acc: 1.0000

Epoch 3/24
----------
train Loss: 0.2497 Acc: 1.0000
val Loss: 0.1130 Acc: 1.0000

Epoch 4/24
----------
train Loss: 0.1876 Acc: 0.9231
val Loss: 0.0479 Acc: 1.0000

Epoch 5/24
----------
train Loss: 0.0870 Acc: 1.0000
val Loss: 0.0161 Acc: 1.0000

Epoch 6/24
----------
train Loss: 0.0510 Acc: 1.0000
val Loss: 0.0047 Acc: 1.0000

Epoch 7/24
----------
train Loss: 0.0737 Acc: 0.9231
val Loss: 0.0011 Acc: 1.0000

Epoch 8/24
----------
train Loss: 0.0363 Acc: 1.0000
val Loss: 0.0004 Acc: 1.0000

Epoch 9/24
----------
train Loss: 0.0443 Acc: 1.0000
val Loss: 0.0002 Acc: 1.0000

Epoch 10/24
----------
train Loss: 0.0084 Acc: 1.0000
val Loss: 0.0001 Acc: 1.0000

Epoch 11/24
----------
train Loss: 0.0141 Acc: 1.0000
val Loss: 0.0001 Acc: 1.0000

Ep

In [7]:
model.load_state_dict(best_model_wts)
torch.save(model.state_dict(), 'best_alexnet.pth')